# Laboratorium 1: Dijkstra & A* & Tabu Search

**Marcin Wawer [272697]**

#### Uwaga - eksportowanie ipynb do pdf, znacząco wpłynęło na formatowanie tekstu, szczególnie na tabele wynikowe i różne oznacznie, co obniża jakość dostarczonego raportu. Rozwiązanie wysyłam również w HTMLu, jak i natywnie w ipynb. Najlepiej przeglądać je w HTMLu, bo wyniki są od razu widoczne, a formatowanie tekstu jest zachowane.

# <span style="color: #915F6D;">Budowa grafów i setup</span>

#### Wykorzystane biblioteki

- `typing` - umożliwia stosowanie adnotacji typów; Final służy do oznaczania stałych.
- `datetime / timedelta` - służy do operacji na datach i czasach, w tym obliczania różnic czasowych.
- `os` - umożliwia manipulację ścieżkami plików i interakcję z systemem operacyjnym.
- `pandas` - ułatwia analizę, modyfikację i formatowanie danych w strukturach DataFrame.
- `csv` - standardowy moduł do odczytu i zapisu plików CSV.
- `heapq` - umożliwia implementację kolejki priorytetowej opartej na kopcu.
- `math` - zawiera funkcje matematyczne przydatne przy obliczeniach.
- `time` - umożliwia pomiar czasu wykonania kodu.
- `random` - umożliwia generowanie liczb losowych i losowe mieszanie list.

In [1]:
import src.Utilities.formatter as formatter
import src.Graph_algorithms.graph_loader as loader
import src.Graph_algorithms.a_star as a_star
import src.Graph_algorithms.dijkstra as dijkstra
import src.Graph_algorithms.tabu_search as tabu
from datetime import datetime
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

## <span style="color: #915F6D;">Stałe (Config/constants.py)</span>

- `BASE_DIR i FILE_NAME` - pomagają w budowaniu poprawnych ścieżek do plików.
- `TIME_COST_PER_SEC oraz CHANGE_COST_PER_CHANGE` - definiują sposób skalowania kosztów czasu oraz przesiadki w algorytmach wyszukiwania trasy.
- `MIN_CHANGE_TIME` - określa minimalny wymagany czas, jaki musi upłynąć, aby przesiadka była akceptowana.

```python 
BASE_DIR: Final[str] = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
FILE_NAME: Final[str] = os.path.join(BASE_DIR, "Data", "connection_graph.csv")

TIME_COST_PER_SEC: Final[int] = 1
CHANGE_COST_PER_CHANGE: Final[int] = 1

MIN_CHANGE_TIME: Final[timedelta] = timedelta(minutes=2)

## <span style="color: #915F6D;">Budowa grafu (Graph_algorithms/graph_loader.py)</span>

#### `parse_extended_time`

Funkcja parse_extended_time jest przeznaczona do przetwarzania ciągów znaków reprezentujących czas, które mogą zawierać godziny przekraczające standardowy zakres 0–23. Przykładowo, ciąg “25:10:00” jest interpretowany jako 01:10:00 kolejnego dnia.

Działanie:

- Funkcja dzieli ciąg znaków na trzy elementy: godziny, minuty i sekundy, używając funkcji split(':') oraz map(int, ...).
- Następnie wykorzystuje funkcję divmod, aby obliczyć, ile pełnych dni przekracza dana liczba godzin (np. 25 godzin daje 1 dzień i 1 godzinę).
- Ustawiany jest bazowy obiekt datetime (tutaj datetime(1900, 1, 1)), do którego dodawane są wyliczone dni, godziny, minuty i sekundy poprzez funkcję timedelta.
- Funkcja zwraca wynikowy obiekt datetime, który prawidłowo reprezentuje przekroczony czas.

```python
def parse_extended_time(time_str):
    h, m, s = map(int, time_str.split(':'))
    extra_days, hour = divmod(h, 24)
    base_date = datetime(1900, 1, 1) 
    return base_date + timedelta(days=extra_days, hours=hour, minutes=m, seconds=s)

#### `load_weighted_graph`

Funkcja load_weighted_graph wczytuje dane z pliku CSV i tworzy graf jako słownik w Pythonie. Kluczami słownika są nazwy przystanków początkowych, a wartościami listy słowników reprezentujących połączenia wychodzące z danego przystanku.

Działanie:

- Wczytywanie danych: Otwiera plik CSV z odpowiednim kodowaniem.
- Parsowanie czasu: Dla każdej linii pliku klucze departure_time oraz arrival_time są przetwarzane przez funkcję parse_extended_time, co pozwala poprawnie obsłużyć czasy przekraczające 24 godziny.
- Konwersja danych: Inne kolumny, takie jak start_stop_lat, start_stop_lon, end_stop_lat i end_stop_lon są konwertowane na typ float, co umożliwia późniejsze operacje.
- Ustalanie wagi: Jeśli kryterium wynosi “time” lub “t”, przypisywana jest waga, która obliczana jest jako różnica pomiędzy czasem przyjazdu i odjazdu. Jeśli kryterium to “change” lub “c”, waga jest ustawiana na 0.
- Budowa grafu: Dla każdego wiersza w pliku, funkcja odczytuje wartość start_stop i dodaje przetworzony rekord do listy połączeń tego przystanku w utworzonym słowniku graph.

```python
def load_weighted_graph(criterion):
    if criterion not in ["time", "t", "change", "c"]:
        raise ValueError("wrong criterion")

    graph = {}
    with open(FILE_NAME, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            try:
                row['departure_time'] = parse_extended_time(row['departure_time'])
                row['arrival_time'] = parse_extended_time(row['arrival_time'])
                row['start_stop_lat'] = float(row['start_stop_lat'])
                row['start_stop_lon'] = float(row['start_stop_lon'])
                row['end_stop_lat'] = float(row['end_stop_lat'])
                row['end_stop_lon'] = float(row['end_stop_lon'])
            except Exception as e:
                continue  

            if criterion in ['time', 't']:
                row['weight'] = (row['arrival_time'] - row['departure_time']).total_seconds
            elif criterion in ['change', 'c']:
                row['weight'] = 0

            start = row['start_stop']
            if start not in graph:
                graph[start] = []
            graph[start].append(row)
    
    return graph

## <span style="color: #915F6D">Metody do formatowania wyników metod (Utilities/formatter.py)</span>

#### `format_schedule_df`

Funkcja format_schedule_df ma na celu przekształcenie listy połączeń, która zawiera informacje o poszczególnych segmentach trasy w uporządkowany DataFrame. Dzięki temu wynik końcowy jest czytelny i wygodny do prezentacji.

Działanie:

- Ustawienia wyświetlania: Funkcja ustawia opcje Pandas, by nie ograniczał szerokości wyświetlanego DataFrame i wyświetlał wszystkie kolumny.
- Tworzenie DataFrame: Na podstawie przekazanego path tworzy DataFrame.
- Obliczanie kosztu: W zależności od przekazanego kryterium („time” lub „change”).
- Formatowanie dat: Kolumny zawierające czasy są formatowane tak, by wyświetlały tylko godziny, minuty i sekundy.
- Wynik: Na końcu funkcja zwraca DataFrame zawierający kolumny: line, departure_time, start_stop, arrival_time, end_stop, cost.

#### `format_tabu_route_df`

Funkcja format_tabu_route_df ma za zadanie połączyć wyniki algorytmu Tabu Search, które są podzielone na segmenty, w jedną spójną listę połączeń, a następnie sformatować tę ścieżkę za pomocą funkcji format_schedule_df.

Działanie:

- Łączenie segmentów: Funkcja iteruje po liście segmentów otrzymanej z algorytmu Tabu Search. Każdy segment jest krotką, gdzie segment_path to lista połączeń między danymi przystankami. Funkcja łączy wszystkie te segmenty w jedną listę full_path.
- Formatowanie ścieżki: Następnie wynikowa lista full_path jest przekazywana do funkcji format_schedule_df, która tworzy na jej podstawie czytelny DataFrame, zgodny z opisanym wcześniej formatem.
- Wynik: Ostatecznie funkcja zwraca sformatowany DataFrame, w którym widoczne są szczegółowo wszystkie połączenia trasy.

# <span style="color: lightblue;">Zadanie 1</span>

**Cel** - Znalezienie najkrótszego połączenia pomiędzy zadanymi przystankami A i B, minimalizując czas lub liczbę przesiadek.

- `Dijkstra` - algorytm Dijkstry jest algorytmem zachłannym, który znajduje najkrótsze ścieżki od wierzchołka źródłowego do wszystkich pozostałych w grafie o nieujemnych wagach krawędzi. Opiera się na zasadzie relaksacji odległości i wykorzystuje kolejkę priorytetową, aby w każdym kroku wybierać wierzchołek o minimalnym kosztach dotychczasowych.

- `A*` - algorytm A* to metodą przeszukiwania heurystycznego, która łączy rzeczywisty koszt przebytej drogi (g(n)) z oszacowaniem pozostałego kosztu do celu (h(n)), tworząc funkcję oceny f(n)=g(n)+h(n). Dzięki odpowiednio dobranej heurystyce A* jest w stanie szybciej koncentrować wyszukiwanie w kierunku celu, co czyni go efektywniejszym niż Dijkstra w wielu zastosowaniach, przy zachowaniu optymalności rozwiązania.

In [2]:
graph_time = loader.load_weighted_graph(criterion='time')
graph_changes = loader.load_weighted_graph(criterion='change')

stop_coords = a_star.load_stop_coords()

Wybieram trzy czasy - rano, środek dnia i noc. Z tych trzech czasów będe korzystał w zadaniu 1 i 2.

In [3]:
start_time_1 = datetime.strptime("08:00:00", "%H:%M:%S")
start_time_2 = datetime.strptime("16:00:00", "%H:%M:%S")
start_time_3 = datetime.strptime("23:30:00", "%H:%M:%S")

## <span style="color: lightblue;">Zadanie 1 - A</span>

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu Dijkstry w oparciu o kryterium czasu.

In [4]:
total_cost, path, run_time = dijkstra.dijkstra_min_time(
    graph_time, 
    "Bociania",
    "Muchobór Wielki",
    start_time_1
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop   cost
  23       08:01:00                           Bociania     08:02:00                              Gęsia 120.00
  23       08:02:00                              Gęsia     08:04:00                         Kwidzyńska 120.00
  23       08:04:00                         Kwidzyńska     08:05:00                         Kętrzyńska  60.00
  23       08:05:00                         Kętrzyńska     08:07:00                            KROMERA 120.00
  23       08:07:00                            KROMERA     08:09:00                  Mosty Warszawskie 120.00
  23       08:09:00                  Mosty Warszawskie     08:10:00                       Daszyńskiego  60.00
  23       08:10:00                       Daszyńskiego     08:12:00                        Nowowiejska 120.00
  23       08:12:00                        Nowowiejska     08:13:00                 Jedności Narodowej  60.00
  23      

In [5]:
total_cost, path, run_time = dijkstra.dijkstra_min_time(
    graph_time, 
    "KOZANÓW (Dokerska)",
    "pl. Bema",
    start_time_2
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_2)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                     start_stop arrival_time                       end_stop   cost
 102       16:01:00             KOZANÓW (Dokerska)     16:03:00                        KOZANÓW 180.00
 102       16:03:00                        KOZANÓW     16:04:00                        Dzielna  60.00
 102       16:04:00                        Dzielna     16:05:00                      Wiślańska  60.00
 102       16:05:00                      Wiślańska     16:07:00                        Kolista 120.00
 103       16:10:00                        Kolista     16:13:00      Wejherowska (Hala Orbita) 360.00
 103       16:13:00      Wejherowska (Hala Orbita)     16:15:00                  Port Popowice 120.00
 103       16:15:00                  Port Popowice     16:17:00                 Park Popowicki 120.00
 103       16:17:00                 Park Popowicki     16:18:00 Wrocław Popowice (17.południk)  60.00
 103       16:18:00 Wrocław Popowice (17.południk)     16:19:00       Długa (ogrod

In [6]:
total_cost, path, run_time = dijkstra.dijkstra_min_time(
    graph_time, 
    "FAT",
    "KSIĘŻE MAŁE",
    start_time_3
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_3)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                 start_stop arrival_time                   end_stop    cost
 125       23:31:00                        FAT     23:32:00                Aleja Pracy  120.00
 125       23:32:00                Aleja Pracy     23:33:00                Inżynierska   60.00
 125       23:33:00                Inżynierska     23:34:00     Kolbuszowska (Stadion)   60.00
 125       23:34:00     Kolbuszowska (Stadion)     23:35:00          Krucza (Mielecka)   60.00
 125       23:35:00          Krucza (Mielecka)     23:36:00                     Krucza   60.00
 125       23:36:00                     Krucza     23:38:00             Pl. Hirszfelda  120.00
 125       23:38:00             Pl. Hirszfelda     23:40:00                Komandorska  120.00
 125       23:40:00                Komandorska     23:42:00                      Arena  120.00
 125       23:42:00                      Arena     23:43:00                        EPI   60.00
 125       23:43:00                        EPI    

#### Wnioski:

- Wykonania funkcji są bardzo szybkie (0.39 - 1.48) różnią się odpowiednio, gdy trasa jest dalsza i trzeba przeszukać większy zakres.
- Widać tutaj oczywiście bardzo dużo przesiadek, co jest zachowaniem spodziewany, ponieważ minimalizujemy czas dojazdu.
- Wyraźne różnice w funkcji kosztu pojawiają się dla tego najpóźniejszego przejazdu, gdzie trzeba korzystać z nocnych linii, które rzadziej kursują, co wiążę się z wydłużonym czasem oczekiwania podczas zmian linii.

## <span style="color: lightblue;">Zadanie 1 - B</span>

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu A* w oparciu o kryterium czasu

`haversine` - funkcja oblicza odległość między dwoma punktami na powierzchni Ziemi przy użyciu wzoru haversine. Zwraca wynik w metrach.

`load_stop_coords` - funkcja wczytuje dane z pliku CSV i tworzy słownik, którego klucze to nazwy przystanków, a wartości to krotki (szerokość geograficzna, długość geograficzna). Dzięki temu umożliwia dostęp do współrzędnych przystanków, co jest wykorzystywane przez algorytm A* do obliczania heurystyki.

`Heurystyka` służy do oszacowania pozostałego kosztu od bieżącego węzła do celu. W implementacji A* funkcja heuristic wykorzystuje metodę haversine do wyliczenia odległości między współrzędnymi danego przystanku a celem. Następnie dzieli tę odległość przez parametr max_speed – co daje oszacowanie minimalnego czasu potrzebnego do pokonania tej odległości przy zadanej maksymalnej prędkości.

Parametr `max_speed` decyduje o tym, jak optymistyczne jest oszacowanie.

In [7]:
total_cost, path, run_time = a_star.a_star_min_time(
    graph_time, 
    stop_coords, 
    "Bociania",
    "Muchobór Wielki",
    start_time_1, 
    max_speed=20
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop   cost
  23       08:01:00                           Bociania     08:02:00                              Gęsia 120.00
  23       08:02:00                              Gęsia     08:04:00                         Kwidzyńska 120.00
  23       08:04:00                         Kwidzyńska     08:05:00                         Kętrzyńska  60.00
  23       08:05:00                         Kętrzyńska     08:07:00                            KROMERA 120.00
  23       08:07:00                            KROMERA     08:09:00                  Mosty Warszawskie 120.00
  23       08:09:00                  Mosty Warszawskie     08:10:00                       Daszyńskiego  60.00
  23       08:10:00                       Daszyńskiego     08:12:00                        Nowowiejska 120.00
  23       08:12:00                        Nowowiejska     08:13:00                 Jedności Narodowej  60.00
  23      

In [8]:
total_cost, path, run_time = a_star.a_star_min_time(
    graph_time, 
    stop_coords, 
    "KOZANÓW (Dokerska)",
    "pl. Bema",
    start_time_2, 
    max_speed=20
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_2)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                     start_stop arrival_time                       end_stop   cost
 102       16:01:00             KOZANÓW (Dokerska)     16:03:00                        KOZANÓW 180.00
 102       16:03:00                        KOZANÓW     16:04:00                        Dzielna  60.00
 102       16:04:00                        Dzielna     16:05:00                      Wiślańska  60.00
 102       16:05:00                      Wiślańska     16:07:00                        Kolista 120.00
 103       16:10:00                        Kolista     16:13:00      Wejherowska (Hala Orbita) 360.00
 103       16:13:00      Wejherowska (Hala Orbita)     16:15:00                  Port Popowice 120.00
 103       16:15:00                  Port Popowice     16:17:00                 Park Popowicki 120.00
 103       16:17:00                 Park Popowicki     16:18:00 Wrocław Popowice (17.południk)  60.00
 103       16:18:00 Wrocław Popowice (17.południk)     16:19:00       Długa (ogrod

In [9]:
total_cost, path, run_time = a_star.a_star_min_time(
    graph_time, 
    stop_coords, 
    "FAT",
    "KSIĘŻE MAŁE",
    start_time_3, 
    max_speed=20
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_3)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                 start_stop arrival_time                   end_stop    cost
 125       23:31:00                        FAT     23:32:00                Aleja Pracy  120.00
 125       23:32:00                Aleja Pracy     23:33:00                Inżynierska   60.00
 125       23:33:00                Inżynierska     23:34:00     Kolbuszowska (Stadion)   60.00
 125       23:34:00     Kolbuszowska (Stadion)     23:35:00          Krucza (Mielecka)   60.00
 125       23:35:00          Krucza (Mielecka)     23:36:00                     Krucza   60.00
 125       23:36:00                     Krucza     23:38:00             Pl. Hirszfelda  120.00
 125       23:38:00             Pl. Hirszfelda     23:40:00                Komandorska  120.00
 125       23:40:00                Komandorska     23:42:00                      Arena  120.00
 125       23:42:00                      Arena     23:43:00                        EPI   60.00
 125       23:43:00                        EPI    

In [10]:
total_cost, path, run_time = a_star.a_star_min_time(
    graph_time, 
    stop_coords, 
    "Bociania",
    "Muchobór Wielki",
    start_time_1, 
    max_speed=30
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop   cost
  23       08:01:00                           Bociania     08:02:00                              Gęsia 120.00
  23       08:02:00                              Gęsia     08:04:00                         Kwidzyńska 120.00
  23       08:04:00                         Kwidzyńska     08:05:00                         Kętrzyńska  60.00
  23       08:05:00                         Kętrzyńska     08:07:00                            KROMERA 120.00
  23       08:07:00                            KROMERA     08:09:00                  Mosty Warszawskie 120.00
  23       08:09:00                  Mosty Warszawskie     08:10:00                       Daszyńskiego  60.00
  23       08:10:00                       Daszyńskiego     08:12:00                        Nowowiejska 120.00
  23       08:12:00                        Nowowiejska     08:13:00                 Jedności Narodowej  60.00
  23      

In [11]:
total_cost, path, run_time = a_star.a_star_min_time(
    graph_time, 
    stop_coords, 
    "Bociania",
    "Muchobór Wielki",
    start_time_1, 
    max_speed=2
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="time", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                           start_stop arrival_time                             end_stop   cost
  23       08:01:00                             Bociania     08:02:00                                Gęsia 120.00
  23       08:02:00                                Gęsia     08:04:00                           Kwidzyńska 120.00
 118       08:06:00                           Kwidzyńska     08:09:00                              KROMERA 300.00
   6       08:11:00                              KROMERA     08:13:00                    Mosty Warszawskie 240.00
   6       08:13:00                    Mosty Warszawskie     08:14:00                         Daszyńskiego  60.00
   6       08:14:00                         Daszyńskiego     08:16:00                          Nowowiejska 120.00
   6       08:16:00                          Nowowiejska     08:17:00                   Jedności Narodowej  60.00
   6       08:17:00                   Jedności Narodowej     08:18:00                   

#### Wnioski:

- Wykonania funkcji są również bardzo szybkie, analogiczne do Dijkstry w przypadku tych określonych parametrów.
- Wyniki funkcji kosztu są takie same jak dla wywołań funkcji Dijkstry, co jest oczekiwannym zachowaniem, wyłączając ostatnie wywołanie.
- Przy niższej wartości max_speed - 2, w ostatnim wywołaniu, heurystyka obliczana jako odległość podzielona przez max_speed przyjmuje większe wartości, co powoduje wyższy całkowity koszt f = g + h. W rezultacie algorytm A* wybiera rozwiązania z wyższym kosztem, w porównaniu z przypadkiem, gdy max_speed jest ustawiony na wyższą wartość - 20, 30.

## <span style="color: lightblue;">Zadanie 1 - C</span>

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu A* w oparciu o kryterium przesiadek

Funkcja koncentruje się wyłącznie na minimalizacji liczby przesiadek podczas wyszukiwania trasy. Zamiast sumować cały czas podróży, algorytm przypisuje koszt oparty na zmianach linii – za każdą wykrytą zmianę dodawany jest stały penalty - CHANGE_COST_PER_CHANGE. Warunek dotyczący minimalnego czasu oczekiwania - MIN_CHANGE_TIME, gwarantuje, że przesiadka jest akceptowana tylko wtedy, gdy użytkownik ma wystarczająco dużo czasu na zmianę połączenia. Funkcja zwraca łączny koszt równy sumie kar za przesiadki, co umożliwia wybór trasy z możliwie najmniejszą liczbą zmian linii, kompletnie niezależnie od całkowitego czasu podróży.

In [12]:
total_cost, path, run_time = a_star.a_star_min_changes(
    graph_changes, 
    "Bociania",
    "Muchobór Wielki", 
    start_time_1
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop  cost
  11       14:44:00                           Bociania     14:45:00                              Gęsia     0
  11       14:45:00                              Gęsia     14:47:00                         Kwidzyńska     0
  11       14:47:00                         Kwidzyńska     14:48:00                         Kętrzyńska     0
  11       14:48:00                         Kętrzyńska     14:50:00                            KROMERA     0
  11       14:50:00                            KROMERA     14:52:00                  Mosty Warszawskie     0
  11       14:52:00                  Mosty Warszawskie     14:53:00                       Daszyńskiego     0
  11       14:53:00                       Daszyńskiego     14:55:00                        Nowowiejska     0
  11       14:55:00                        Nowowiejska     14:56:00                 Jedności Narodowej     0
  11       14:56:00

In [13]:
total_cost, path, run_time = a_star.a_star_min_changes(
    graph_changes, 
    "KOZANÓW (Dokerska)",
    "pl. Bema",  
    start_time_2
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_2)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                           start_stop arrival_time                             end_stop  cost
  12       16:07:00                   KOZANÓW (Dokerska)     16:08:00                           Kozanowska     0
  12       16:08:00                           Kozanowska     16:09:00                                Modra     0
  12       16:09:00                                Modra     16:10:00                    PILCZYCKA (Anima)     0
  12       16:10:00                    PILCZYCKA (Anima)     16:11:00                              Kolista     0
  12       16:11:00                              Kolista     16:16:00                               Kwiska     0
  12       16:16:00                               Kwiska     16:17:00                         Małopanewska     0
  12       16:17:00                         Małopanewska     16:19:00                         Niedźwiedzia     0
  12       16:19:00                         Niedźwiedzia     16:21:00        Wrocław Mikołajów (

In [14]:
total_cost, path, run_time = a_star.a_star_min_changes(
    graph_changes, 
    "FAT",
    "KSIĘŻE MAŁE", 
    start_time_3
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_3)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                 start_stop arrival_time                   end_stop  cost
 143       23:47:00                        FAT     23:48:00    GRABISZYŃSKA (Cmentarz)     0
 143       23:48:00    GRABISZYŃSKA (Cmentarz)     23:49:00 GRABISZYŃSKA (Cmentarz II)     0
 143       23:49:00 GRABISZYŃSKA (Cmentarz II)     23:50:00                     OPORÓW     0
 143       23:50:00                     OPORÓW     23:51:00                  Solskiego     0
 143       23:51:00                  Solskiego     23:52:00                    Wiejska     0
 143       23:52:00                    Wiejska     23:53:00                   Kadłubka     0
 143       23:53:00                   Kadłubka     23:54:00                     Stanki     0
 143       23:54:00                     Stanki     23:54:00                Bukowskiego     0
 143       23:54:00                Bukowskiego     23:55:00                 RACŁAWICKA     0
 143       23:55:00                 RACŁAWICKA     23:56:00           

#### Wnioski:
- Wywołanie funkcji z analogicznymi parametrami zajmuje znacznie więcej czasu, w przypadku trasy Muchobór Wielki - Kowale, względem wywołania alogrytmu minimalizującego czas.
- Przez to, że algorytm skupia się tylko na minimalizowaniu liczby przesiadek, niektóre czasy przejazdu przy testach były bardzo długie, co nie jest niespodziewane, ponieważ algorytm skupia się tylko i wyłącznie na przesiadkach. Myślę, że dobrze byłoby rozszerzyć te funkcję, aby brała pod uwagę również koszt czasowy i odpowiednio dostosować wtedy ich wagi.
- Natomiast widać tutaj efektywne działanie funkcji - wydłuża czas tras, ale drastycznie zmniejsza ilość przesiadek, względem funkcji minimalizujących czas.

## <span style="color: lightblue;">Zadanie 1 - D</span>

Modyfikacja algorytmu A* z punktów (b) lub (c), który pozwoli na zmniejszenie wartości funkcji kosztu uzyskanego rozwiązania lub czasu obliczeń.

`Beam search` to modyfikacja algorytmu A*, która zamiast rozpatrywać wszystkie możliwe ruchy dla bieżącego rozwiązania, wybiera jedynie losową próbkę ruchów do oceny. Dzięki temu liczba sprawdzanych wariantów jest znacząco mniejsza, co skraca czas obliczeń i zmniejsza obciążenie pamięciowe, zwłaszcza przy dużej liczbie przystanków. Pomimo że ograniczenie to może potencjalnie wpłynąć na optymalność znalezionego rozwiązania, odpowiednio dobrany rozmiar wyszukiwania pozwala na zachowanie dobrej jakości wyniku.

In [15]:
total_cost, path, run_time = a_star.a_star_min_changes_beam(
    graph_changes, 
    "Bociania",
    "Muchobór Wielki", 
    start_time_1, 
    beam_width=5000
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop  cost
  11       14:44:00                           Bociania     14:45:00                              Gęsia     0
  11       14:45:00                              Gęsia     14:47:00                         Kwidzyńska     0
  11       14:47:00                         Kwidzyńska     14:48:00                         Kętrzyńska     0
  11       14:48:00                         Kętrzyńska     14:50:00                            KROMERA     0
  11       14:50:00                            KROMERA     14:52:00                  Mosty Warszawskie     0
  11       14:52:00                  Mosty Warszawskie     14:53:00                       Daszyńskiego     0
  11       14:53:00                       Daszyńskiego     14:55:00                        Nowowiejska     0
  11       14:55:00                        Nowowiejska     14:56:00                 Jedności Narodowej     0
  11       14:56:00

In [24]:
total_cost, path, run_time = a_star.a_star_min_changes_beam(
    graph_changes, 
    "Bociania",
    "Muchobór Wielki", 
    start_time_1, 
    beam_width=1000
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

route not found


In [17]:
total_cost, path, run_time = a_star.a_star_min_changes_beam(
    graph_changes, 
    "Bociania",
    "Muchobór Wielki", 
    start_time_1, 
    beam_width=100
)

if path is not None:
    df = formatter.format_schedule_df(path, criterion="change", start_time=start_time_1)  
    print(df.to_string(index=False) + "\n")

    data = {
        "metric": ["TOTAL COST", "RUN TIME"],
        "value": [total_cost, run_time]
    }
    df_summary = pd.DataFrame(data)
    print(df_summary.to_string(index=False))
else:
    print("route not found")

line departure_time                         start_stop arrival_time                           end_stop  cost
  11       14:44:00                           Bociania     14:45:00                              Gęsia     0
  11       14:45:00                              Gęsia     14:47:00                         Kwidzyńska     0
  11       14:47:00                         Kwidzyńska     14:48:00                         Kętrzyńska     0
  11       14:48:00                         Kętrzyńska     14:50:00                            KROMERA     0
  11       14:50:00                            KROMERA     14:52:00                  Mosty Warszawskie     0
  11       14:52:00                  Mosty Warszawskie     14:53:00                       Daszyńskiego     0
  11       14:53:00                       Daszyńskiego     14:55:00                        Nowowiejska     0
  11       14:55:00                        Nowowiejska     14:56:00                 Jedności Narodowej     0
  11       14:56:00

#### Wnioski:
- Beam Search działa zgodnie z założeniami - przy dużym rozmiarze wyszukiwania znajduje optymalne rozwiązanie - 1 przesiadka, ale trwa to odpowiednio długo. Natomiast pomniejszając rozmiar wyszukiwań czas działania algorytmu drastycznie spada, ale znaleziony wynik jest daleki od optymalnego - 6 przesiadek.
- Dla niektórych dobranych rozmiarów wyszukiwań zdaża się rownież, że algorytm w ogóle nie znajduje rozwiązań, tam gdzie zwykły A* takowe znajdował.
- Beam Search wydaje się ciekawą i efektywną modyfikacją, natomiast bardzo istotne jest odpowiednie dobranie tego rozmiaru, co nie jest łatwym zadaniem.

# <span style="color: lightgreen;">Zadanie 2</span>

**Cel** - Znalezienie najkrótszego połączenia dla przystanku początkowego A oraz listy przystanków L=A2, ..., An, przebiegającego przez wszystkie przystanki z L i wracającą do A., minimalizując czas lub liczbę przesiadek.

- `Tabu Search` - metaheurystyka optymalizacyjna, która unika utknięcia w lokalnych minimach poprzez zapamiętywanie ostatnio odwiedzanych rozwiązań i ograniczenie powtarzania tych samych ruchów

## <span style="color: lightgreen;">Zadanie 2 - A</span>

`calculate_route_cost` - funkcja przyjmuje permutację przystanków - route, oraz dane o grafie, a następnie oblicza łączny koszt trasy. Koszt poszczególnego segmentu trasy jest wyznaczany przez funkcję kosztu (cost_func) i zależy od wybranego kryterium.

`tabu_search_route` - algorytm rozpoczyna od losowej permutacji przystanków do odwiedzenia i iteracyjnie generuje sąsiedztwo rozwiązań – poprzez zamianę par przystanków. Każdy ruch oceniany jest za pomocą funkcji calculate_route_cost. Aby uniknąć wielokrotnego rozpatrywania tych samych ruchów, wykorzystana jest tabu_list, która przechowuje już rozpatrzone zamiany. 

Jako funkcje kosztu dla poniższych algorytmu będe korzystał z algorytmu Dijkstry do minimalizowania czasu i algorytmu A* do minimalizowania przesiadek. Jako czas będę podawał start_time_1, czyli 8:00.

In [18]:
cost_func = dijkstra.dijkstra_min_time

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route(
    "Bociania", 
    ["Rynek", "FAT", "Ogród Botaniczny", "Muchobór Wielki"],
    start_time_1, 
    graph_time, 
    cost_func, 
    criterion="time", 
    iterations=1000
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="time", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                         start_stop arrival_time                           end_stop   cost
  23       08:01:00                           Bociania     08:02:00                              Gęsia 120.00
  23       08:02:00                              Gęsia     08:04:00                         Kwidzyńska 120.00
  23       08:04:00                         Kwidzyńska     08:05:00                         Kętrzyńska  60.00
  23       08:05:00                         Kętrzyńska     08:07:00                            KROMERA 120.00
  23       08:07:00                            KROMERA     08:09:00                  Mosty Warszawskie 120.00
  23       08:09:00                  Mosty Warszawskie     08:10:00                       Daszyńskiego  60.00
  23       08:10:00                       Daszyńskiego     08:12:00                        Nowowiejska 120.00
  23       08:12:00                        Nowowiejska     08:13:00                 Jedności Narodowej  60.00
  23      

In [19]:
cost_func = a_star.a_star_min_changes

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route(
    "Bociania", 
    ["Rynek", "FAT", "Ogród Botaniczny", "Muchobór Wielki"],
    start_time_1, 
    graph_changes, 
    cost_func, 
    criterion="change", 
    iterations=1000
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="change", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                           start_stop arrival_time                             end_stop  cost
  11       14:44:00                             Bociania     14:45:00                                Gęsia     0
  11       14:45:00                                Gęsia     14:47:00                           Kwidzyńska     0
  11       14:47:00                           Kwidzyńska     14:48:00                           Kętrzyńska     0
  11       14:48:00                           Kętrzyńska     14:50:00                              KROMERA     0
  11       14:50:00                              KROMERA     14:52:00                    Mosty Warszawskie     0
  11       14:52:00                    Mosty Warszawskie     14:53:00                         Daszyńskiego     0
  11       14:53:00                         Daszyńskiego     14:55:00                          Nowowiejska     0
  11       14:55:00                          Nowowiejska     14:56:00                   Jedności

#### Wnioski:

- Czas działania algorytmów jest znacznie dłuszy, niż w przypadku zadania 1 - raczej spodziewane i normalne zachowanie, którego można było się spodziewać.
- Algorytm minimalizujący liczbę przesiadek zabiera znacznie więcej czasu niż ten, który minimalizuje czas jazdy.
- Widać tutaj to o czym mówiłem wcześniej, czyli trasa, która minimalizuje przesiadki trwa absurdalnie długo od 8:00 do 21:34, co jest zachowaniem spodziewanym, bo algorytm nie zwraca uwagi na ilość czasu, interesują go tylko przesiadki. Dlatego wracając do mojego poprzedniego wniosku, pewnie warto byłoby dostosować algorytm w taki sposób, aby zwracał również w pewnym stopniu czas trasy.
- Oba wywołania (min. przesiadek, min. czasu) poprawnie zwracają trasę, odwiedzając wszystkie zadane przystanki i wracając do przystanku początkowego.

## <span style="color: lightgreen;">Zadanie 2 - B</span>

`tabu_search_route_dynamic_size` wykorzystuje dynamiczny rozmiar tabu listy – w moim przypadku, ustawiany jako dwukrotność liczby przystanków, ale można łatwo to zmienić. Ma to na celu ograniczyć liczbę sprawdzanych ruchów w każdej iteracji, aby skrócić czas obliczeń.

In [20]:
cost_func = a_star.a_star_min_changes

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route_dynamic_size(
    "Bociania", 
    ["Rynek", "FAT", "Ogród Botaniczny", "Muchobór Wielki"],
    start_time_1, 
    graph_changes, 
    cost_func, 
    criterion="change", 
    iterations=1000
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="change", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                           start_stop arrival_time                             end_stop  cost
  11       14:44:00                             Bociania     14:45:00                                Gęsia     0
  11       14:45:00                                Gęsia     14:47:00                           Kwidzyńska     0
  11       14:47:00                           Kwidzyńska     14:48:00                           Kętrzyńska     0
  11       14:48:00                           Kętrzyńska     14:50:00                              KROMERA     0
  11       14:50:00                              KROMERA     14:52:00                    Mosty Warszawskie     0
  11       14:52:00                    Mosty Warszawskie     14:53:00                         Daszyńskiego     0
  11       14:53:00                         Daszyńskiego     14:55:00                          Nowowiejska     0
  11       14:55:00                          Nowowiejska     14:56:00                   Jedności

#### Wnioski:

- Ta modyfikacja Tabu Searcha w moim przypadku poprawia czas obliczeń, natomiast bardzo nieznacznie.
- Być może można byłoby to ulepszyć, lepiej manipulując tym dynamicznym rozmiarem tablicy, natomiast przy zadanych atrybutach poprawa jest, ale nie jest imponująca. Również dla dłuższej trasy, poprawa byłaby z pewnością bardziej znaczna.

## <span style="color: lightgreen;">Zadanie 2 - C</span>

`tabu_search_route_aspiration_rule` - funkcja wprowadza mechanizm aspiracji, który pozwala zaakceptować ruch znajdujący się na liście tabu, o ile jego koszt jest niższy niż dotychczasowy najlepszy koszt. Dzięki temu, nawet jeśli dany ruch był wcześniej zakazany, jest on rozpatrywany, gdy prowadzi do lepszego rozwiązania. Pozwala to na dynamiczne i bardziej elastyczne przeszukiwanie przestrzeni rozwiązań oraz potencjalne znalezienie globalnie lepszego wyniku, eliminując ryzyko utknięcia w lokalnych minimach.

In [21]:
cost_func = a_star.a_star_min_changes

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route_aspiration_rule(
    "Bociania", 
    ["Rynek", "FAT", "Ogród Botaniczny", "Muchobór Wielki"],
    start_time_1, 
    graph_changes, 
    cost_func, 
    criterion="change", 
    iterations=1000
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="change", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                           start_stop arrival_time                             end_stop  cost
  11       14:44:00                             Bociania     14:45:00                                Gęsia     0
  11       14:45:00                                Gęsia     14:47:00                           Kwidzyńska     0
  11       14:47:00                           Kwidzyńska     14:48:00                           Kętrzyńska     0
  11       14:48:00                           Kętrzyńska     14:50:00                              KROMERA     0
  11       14:50:00                              KROMERA     14:52:00                    Mosty Warszawskie     0
  11       14:52:00                    Mosty Warszawskie     14:53:00                         Daszyńskiego     0
  11       14:53:00                         Daszyńskiego     14:55:00                          Nowowiejska     0
  11       14:55:00                          Nowowiejska     14:56:00                   Jedności

#### Wnioski:

- Ta modyfikacja Tabu Search u mnie zdecydowanie wydłuża czas obliczeń.
- Algorytm znajduje optimum dla zadanej trasy, podobnie jak poprzednie wersji, więc pomimo mechanizmu aspiracji nie zmniejsza to funkcji kosztu.
- Prawdopodbnie ta modyfikacja algroytmu spełniała by swoje zadanie i minimalizowała jeszcze bardziej funkcję kosztu względem poprzednich wersji, gdyby trasa była znacznie dłuższa. 

## <span style="color: lightgreen;">Zadanie 2 - D</span>

`tabu_search_route_with_sampling` - funkcja ta wprowadza modyfikacje klasycznego algorytmu Tabu Search, która wprowadza strategię próbkowania sąsiedztwa. Zamiast rozpatrywać wszystkie możliwe zamiany przystanków, co przy dużej liczbie elementów znacząco wydłuża czas obliczeń, funkcja losowo wybiera jedynie pewien procent możliwych ruchów - określany przez parametr sample_ratio.

In [22]:
cost_func = a_star.a_star_min_changes

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route_with_sampling(
    "Muchobór Wielki", 
    ["Rynek", "FAT", "Bociania"],
    start_time_1, 
    graph_changes, 
    cost_func, 
    criterion="change", 
    iterations=1000,
    sample_ratio=0.9
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="change", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                           start_stop arrival_time                             end_stop  cost
 132       08:01:00                      Muchobór Wielki     08:02:00           Muchobór Wielki (Roślinna)     0
 132       08:02:00           Muchobór Wielki (Roślinna)     08:03:00                             Tyrmanda     0
 132       08:03:00                             Tyrmanda     08:04:00      MIŃSKA (Rondo Rotm. Pileckiego)     0
 132       08:04:00      MIŃSKA (Rondo Rotm. Pileckiego)     08:06:00                       Rogowska (P+R)     0
 132       08:06:00                       Rogowska (P+R)     08:08:00              Strzegomska (krzyżówka)     0
 132       08:08:00              Strzegomska (krzyżówka)     08:09:00                          Nowodworska     0
 132       08:09:00                          Nowodworska     08:10:00                      Strzegomska 148     0
 132       08:10:00                      Strzegomska 148     08:11:00                           

In [23]:
cost_func = a_star.a_star_min_changes

best_cost, best_solution, best_segments, run_time = tabu.tabu_search_route_with_sampling(
    "Muchobór Wielki", 
    ["Rynek", "FAT", "Bociania"],
    start_time_1, 
    graph_changes, 
    cost_func, 
    criterion="change", 
    iterations=1000,
    sample_ratio=0.4
)

df_route = formatter.format_tabu_route_df(
    best_segments, 
    criterion="change", 
    start_time=start_time_1
)

print(df_route.to_string(index=False) + "\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME", "OPTIMAL ORDER OF VISITING STOPS"],
    "value": [best_cost, run_time, best_solution]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                           start_stop arrival_time                             end_stop  cost
 132       08:01:00                      Muchobór Wielki     08:02:00           Muchobór Wielki (Roślinna)     0
 132       08:02:00           Muchobór Wielki (Roślinna)     08:03:00                             Tyrmanda     0
 132       08:03:00                             Tyrmanda     08:04:00      MIŃSKA (Rondo Rotm. Pileckiego)     0
 132       08:04:00      MIŃSKA (Rondo Rotm. Pileckiego)     08:06:00                       Rogowska (P+R)     0
 132       08:06:00                       Rogowska (P+R)     08:08:00              Strzegomska (krzyżówka)     0
 132       08:08:00              Strzegomska (krzyżówka)     08:09:00                          Nowodworska     0
 132       08:09:00                          Nowodworska     08:10:00                      Strzegomska 148     0
 132       08:10:00                      Strzegomska 148     08:11:00                           

#### Wnioski:

- Zgodnie z założeniami przy ustawieniu np. sample_ratio = 0.9, algorytm działa dłużej, ale znajduje optymalne rozwiązanie.
- Gdy ustawimy sample_ratio = 0.4, algorytm działa 2x szybciej, natomiast znajduje gorsze rozwiązanie (4 przesiadki do 3).
- Można fajnie manipulować tym współczynnikiem, aby odpowiednio dostosować koszt obliczeń do stopnia szansy na znalezienie minimalnego rozwiązania.

# <span style="color: Salmon;">Podsumowanie</span>

W opracowanym rozwiązaniu zastosowałem różne algorytmy oraz ich modyfikacje umożliwiające efektywne wyznaczanie optymalnych tras w grafie reprezentującym rozkład jazdy komunikacji miejskiej. Kluczowe elementy zadania:

- Wczytywanie danych i parsowanie czasu: dzięki funkcji parse_extended_time możliwe jest poprawne przetwarzanie nietypowych formatów czasu (np. “25:10:00”), co pozwala na uwzględnienie tras przekraczających standardowy 24-godzinny cykl.
- Funkcje kosztu i analiza trasy: funkcja calculate_route_cost oblicza łączny koszt trasy dla wybranej permutacji przystanków. W zależności od kryterium koszt może być obliczany jako suma rzeczywistego czasu jazdy i oczekiwania (z wagą TIME_COST_PER_SEC) lub jako suma kar (z wagą CHANGE_COST_PER_CHANGE) dodawanych przy zmianie linii. Oprócz tego, funkcja weryfikuje, czy czas oczekiwania przy przesiadkach spełnia warunek minimalny określony przez MIN_CHANGE_TIME – w przeciwnym razie trasa jest odrzucana.
- Algorytm Tabu Search: w celu przeszukiwania permutacji odwiedzanych przystanków wykorzystałem algorytm Tabu Search. Dzięki zastosowaniu tabu list algorytm unika cyklicznego przeszukiwania i zbliża się do globalnego minimum funkcji kosztu. Mechanizmy aspiracji umożliwiają zaakceptowanie ruchów tabu, jeśli prowadzą one do lepszego wyniku niż dotychczasowe rozwiązania.
- Modyfikacja z próbkowaniem sąsiedztwa: w celu skrócenia czasu obliczeń dodałem również rozszerzenie algorytmu – Tabu Search z dynamicznym próbkowaniem sąsiedztwa. Dzięki parametrowi sample_ratio, w każdej iteracji rozpatrywane są tylko losowo wybrane ruchy z pełnej przestrzeni sąsiedztwa. To rozwiązanie znacząco redukuje czas przeszukiwania, jednocześnie zapewniając dobrą jakość znalezionych rozwiązań.
- Parametry systemowe: w zadaniu kluczowe są również parametry takie jak TIME_COST_PER_SEC, CHANGE_COST_PER_CHANGE, MIN_CHANGE_TIME oraz max_speed, które wpływają na funkcję kosztu oraz heurystykę w algorytmie A*. Wartości te umożliwiają skalowanie kosztu podróży lub przesiadek i dostosowanie algorytmu do specyfiki problemu i rozwiązania, którego oczekujemy.
- W mojej implementacji zdecydowałem się dodawać nowe wersje algorytmów z określonymi modyfikacjami, zamiast modyfikować jeden algorytm. To pozwoliło mi na łatwiejsze porównania i testowanie algorytmów.

Napotkane problemy podczas implementacji:

- Początkowo nie zauwazyłem w pliku, że czas może wykraczać poza 24h i to oznacza, że tramwaj jest następnego dnia po północy. Gdy to zauważyłem i odpowiednio poprawiłem ładowanie grafu funkcja dla późnych godzin przejazdów również działa poprawnie.
- W pierwszej wersji rozwiązania zapomniałem o uwzględnieniu czasu oczekiwania między przesiadkami oraz tego, że nie zawsze istnieje tramwaj, który odjeżdża idealnie o zadanej godzinie, co prowadziło do mniejszego kosztu, niezgodnego z rzeczywistością. 
- Metoda prób i błędów w celu dobrania odpowiednich tras do prezentacji, które jednocześnie nie będą bardzo długo się wykonywać, ale jednocześnie będą w stanie pokazać omawiane różnice i modyfikację algorytmów.
- Na początku zignorowałem również czas potrzebny na przesiadkę, co prowadziło do sytuacji, gdzie czas na przesiadkę był równy 0 sekund, co jest oczywiście nierealne w rzeczywistości, natomiast odpowiednio szybko zorientowałem się, że coś jest nie tak, a same algorytmy potrzebowały tylko marginalnych zmian, aby to zaimplementować.

Wagi dla kosztu czasowego i kosztu przesiadki:

- Zdecydowałem się uwzględnić mnożnik dla kosztu czasu i przesiadek, poprzez stałe w pliku Config/constants.py.
- W samym rozwiązaniu tego zadania nie ma on za dużego udziału, ponieważ minimalizujemy albo czas trasy, albo liczbę przesiadek. Natomiast przy ewentulanym rozwinięciu algorytmów można byłoby uwzględnić i czas trasy i liczbę przesiadek. Wtedy odpowiednia manipulacjami wagą pozwoliłaby uzyskać zadowalające rozwiązanie, zależnie od naszego celu.
- Czas który potrzebny jest na przesiadkę ustawiony jest u mnie na 2 minuty.

Wydajność:

- W pierwszym zadaniu czasy wykonywanie algorytmów są raczej bardzo krótkie.
- W drugim zadaniu czasy te znacznie rosną, co jest zdecydowanie spodziewanym zachowaniem. Natomiast nie myślałem, że te czasy wzrosną, aż tak bardzo, pomaga z tym natomiast modyfikacja w podpunkcie d, którą już wcześniej opisywałem.
- Ogólnie sporą część czasu zajmuje samo ładowanie danych do grafu, w przypadku pierwszego zadania, jest to operacja znacznie bardziej kosztowna w porównaniu do wywołania algorytmów. Natomiast załadować danę do grafu wystarczy tylko raz na początku, a później z gotowym grafem można już działać wielokrotnie.
- Python nie jest również najszybszym językiem, rozwiązanie tego zadania np. w C lub C++, z pewnością wpłynęłoby pozytywnie na wydajność.

Podsumowując, zadanie było ciekawym doświadczeniem pracy z wyżej wymienionymi algorytmami, a znajdowanie tras pomiędzy przystankami, którymi jeździ się na codzień na żywo oraz porównywanie uzyskanych wyników z aplikacją Jakdojade sprawiło mi całkiem sporo zabawy.